# Exercício 1

Resolva o problema de transportes apresentado abaixo, no qual estão apresentados os custos unitários de transportes (em R$/ton), as demandas (em ton/mês) e as ofertas (em ton/mês).

|               	| **Mercado 1** 	| **Mercado 2** 	| **Mercado 3** 	| **Mercado 4** 	| **Oferta** 	|
|:-------------:	|:-------------:	|:-------------:	|:-------------:	|:-------------:	|:----------:	|
| **Fábrica 1** 	|     10,00     	|      7,00     	|      5,00     	|      6,00     	|   220,00   	|
| **Fábrica 2** 	|     12,00     	|      7,00     	|      6,00     	|      4,00     	|   180,00   	|
| **Fábrica 3** 	|     13,00     	|      6,00     	|      3,00     	|      5,00     	|   230,00   	|
|  **Demanda**  	|     150,00    	|     165,00    	|     210,00    	|     90,00     	|            	|

# Solução

In [2]:
# --- Importando bibliotecas --- #
import pyomo.environ as pyo
import pandas as pd

In [3]:
# --- Dados de entrada --- #
ofertas = {'Fabrica 1': 220, 'Fabrica 2': 180, 'Fabrica 3': 230} # Ofertas de cada fábrica
demandas = {'Mercado 1': 150, 'Mercado 2': 165, 'Mercado 3': 210, 'Mercado 4': 90} # Demandas de cada mercado
# Custo de transporte de cada fábrica para cada mercado
custos = {('Fabrica 1', 'Mercado 1'): 10, ('Fabrica 1', 'Mercado 2'): 7, ('Fabrica 1', 'Mercado 3'): 5, ('Fabrica 1', 'Mercado 4'): 6,
          ('Fabrica 2', 'Mercado 1'): 12, ('Fabrica 2', 'Mercado 2'): 7, ('Fabrica 2', 'Mercado 3'): 6, ('Fabrica 2', 'Mercado 4'): 4,
          ('Fabrica 3', 'Mercado 1'): 13, ('Fabrica 3', 'Mercado 2'): 6, ('Fabrica 3', 'Mercado 3'): 3, ('Fabrica 3', 'Mercado 4'): 5
          }

In [4]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel() # Criação de um objeto para o modelo

In [5]:
# --- Declaração dos Conjuntos de Iteração --- #
modelo.mercados = pyo.Set(initialize=demandas.keys()) # Conjunto de mercados
modelo.fabricas = pyo.Set(initialize=ofertas.keys()) # Conjunto de fábricas

In [6]:
# --- Declaração de Parâmetros --- #
modelo.custo_transporte = pyo.Param(modelo.fabricas, modelo.mercados, initialize=custos) # Custo de transporte
modelo.demanda = pyo.Param(modelo.mercados, initialize=demandas) # Demanda de cada mercado
modelo.oferta = pyo.Param(modelo.fabricas, initialize=ofertas) # Oferta de cada fábrica

In [7]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.fabricas, modelo.mercados, within=pyo.NonNegativeReals) # Quantidade a ser transportada de cada fábrica para a cada mercado

In [8]:
# --- Declaração da Função Objetivo --- #
def computar_custo_de_transporte(modelo):
    '''
        Recebe um objeto de modelo Pyomo e retorna o custo total de transporte da origem i para o destino j.
    '''
    custo_de_transporte = sum(
        modelo.custo_transporte[i, j] * modelo.x[i, j]
        for i in modelo.fabricas
        for j in modelo.mercados
    )
    return custo_de_transporte

modelo.objetivo = pyo.Objective(rule=computar_custo_de_transporte, sense=pyo.minimize) # Função objetivo

In [9]:
# --- Declaração das Restrições --- #
def regra_capacidade(modelo, i):
    '''
    Recebe um objeto de modelo Pyomo e um índice i e retorna a expressão de restrição de capacidade da fábrica i.
    '''
    return sum(modelo.x[i, :]) <= modelo.oferta[i]
# --- 
modelo.capacidade_rest = pyo.Constraint(modelo.fabricas, rule=regra_capacidade) # Restrição de capacidade
# --- #
def regra_demanda(modelo, j):
    '''
    Recebe um objeto de modelo Pyomo e um índice j e retorna a expressão de restrição de demanda do mercado j.
    '''
    return sum(modelo.x[:, j]) == modelo.demanda[j]
# ---
modelo.demanda_rest = pyo.Constraint(modelo.mercados, rule=regra_demanda) # Restrição de demanda

In [10]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi') # Criação de um objeto solver
solver.solve(modelo) # Resolução do modelo

{'Problem': [{'Name': 'x1', 'Lower bound': 3625.0, 'Upper bound': 3625.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': 24, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.421825647354126}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [11]:
# --- Extração dos Resultados --- #
dados_transporte = [{'de': i, 'para': j, 'quantidade': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_transporte).pivot(index='de', columns='para', values='quantidade')
resultados.to_excel('_ex_01_resultados_modelo_exato.xlsx')

In [12]:
print(f'Função Objetivo: {modelo.objetivo()}')

Função Objetivo: 3625.0


In [13]:
dados_transporte

[{'de': 'Fabrica 1', 'para': 'Mercado 1', 'quantidade': 150.0},
 {'de': 'Fabrica 1', 'para': 'Mercado 2', 'quantidade': 70.0},
 {'de': 'Fabrica 1', 'para': 'Mercado 3', 'quantidade': 0.0},
 {'de': 'Fabrica 1', 'para': 'Mercado 4', 'quantidade': 0.0},
 {'de': 'Fabrica 2', 'para': 'Mercado 1', 'quantidade': 0.0},
 {'de': 'Fabrica 2', 'para': 'Mercado 2', 'quantidade': 75.0},
 {'de': 'Fabrica 2', 'para': 'Mercado 3', 'quantidade': 0.0},
 {'de': 'Fabrica 2', 'para': 'Mercado 4', 'quantidade': 90.0},
 {'de': 'Fabrica 3', 'para': 'Mercado 1', 'quantidade': 0.0},
 {'de': 'Fabrica 3', 'para': 'Mercado 2', 'quantidade': 20.0},
 {'de': 'Fabrica 3', 'para': 'Mercado 3', 'quantidade': 210.0},
 {'de': 'Fabrica 3', 'para': 'Mercado 4', 'quantidade': 0.0}]